# Viewing Pre-Defined Models in TensorFlow

TensorFlow has some nifty features that make it easy to export and import models into your code. This can be useful to quickly retrain models, explore other people's work, and chain together separate models. To load in a saved graph, we can use a `tf.GraphDef` to help us out.

## What is a GraphDef?

A `GraphDef` is a TensorFlow class dedicated to storing a serialized form of a TensorFlow `Graph`. Once a `GraphDef` is loaded (either from disk or from the `Graph.as_graph_def()` function), you can transfer its data into a graph by using the `tf.import_graph_def()` function.

In order to read/write `GraphDef` objects from disk, you use a compiled protocol buffer file. To read in the file, you use the `GraphDef.ParseFromString()` method, which takes in a byte string object. For example:

```python
with open('my_graph.pb', 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
```
 
To write the file to disk, you use the `GraphDef.SerializeToString()` method:

```python
with open('new_graph.pb', 'wb') as f:
    graph_def = my_existing_graph.as_graph_def()
    f.write(graph_def.SerializeToString())
```


### ...and what is a Protocol Buffer?

[Protocol Buffers](https://developers.google.com/protocol-buffers/), sometimes shortened to protobufs, are an extensible, structured way of communicating data. They're in the same family as XML or JSON, but with some extra features. One such feature is the ability to compile it to binary and read it back with included open source software. It's backed by Google, who are planning on using it for all of their APIs in the near future.

---

# Inspecting Inception

Google's "Inception" models are a family of _very_ deep convolutional neural networks, designed to be competitors in the [ImageNet Large Scale Visual Recognition Challenge competition](http://image-net.org/challenges/LSVRC/). You can read more about their history with these brief papers:

* [Inception v1: GoogLeNet](http://www.cs.unc.edu/~wliu/papers/GoogLeNet.pdf)
* [Inception v2 & v3](https://arxiv.org/pdf/1512.00567v3.pdf)

The architecture of this Inception v3 is shown below:

[![Inception Architecture](images/inception_architecture.png)](images/inception_architecture.png)

You can see that it's a long series of various convolutions and pools before getting to the final few steps. Google's been kind enough to provide a pre-trained version of Inception v3 in TensorFlow, so we're going to load it up and visualize the graph using TensorFlow's built-in TensorBoard utility.

# Downloading and Extracting the Graph Definition

First we need to download the graph definition protocol buffer file. To do so, we're using a nifty function that downloads and extracts a zip file for you if it doesn't already exist.

In [1]:
# Python > 3, TensorFlow > 1.0
import os.path
from six.moves import urllib
import tarfile
import sys

In [2]:
# Modified function from here
# https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/image_retraining/retrain.py

def maybe_download_and_extract(data_url, dest_directory):
    """Download and extract model tar file.
    If the pretrained model we're using doesn't already exist, this function
    downloads it from the TensorFlow.org website and unpacks it into a directory.
    """
    if not os.path.exists(dest_directory):
        os.makedirs(dest_directory)
    filename = data_url.split('/')[-1]
    filepath = os.path.join(dest_directory, filename)
    if not os.path.exists(filepath):

        def _progress(count, block_size, total_size):
            sys.stdout.write('\r>> Downloading %s %.1f%%' %
                            (filename,
                             float(count * block_size) / float(total_size) * 100.0))
        sys.stdout.flush()

        filepath, _ = urllib.request.urlretrieve(data_url,
                                             filepath,
                                             _progress)
        print()
        statinfo = os.stat(filepath)
        print('Successfully downloaded', filename, statinfo.st_size, 'bytes.')
    tarfile.open(filepath, 'r:gz').extractall(dest_directory)

In [3]:
# Specify where to download from
imagenet_url = 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'
# Specify where to save and extract to
dest_directory = './saved_models'

In [7]:
# Actually download/extract the file!
maybe_download_and_extract(imagenet_url, dest_directory)

# Loading in the GraphDef

Now that we've got the graph definition protobuf saved to disk (by default, it's in `inception/classify_image_graph_def.pb`), we can open the file and import it to a `tf.Graph` object.

_Note: we're using a `gfile.FastGFile()` utility here instead of Python's built in `open()` function, but the code would work the same if you swapped in `open()`._

In [8]:
import os.path
import tensorflow as tf
from tensorflow.python.platform import gfile

graph_filename = os.path.join(dest_directory, 'classify_image_graph_def.pb')
graph = tf.Graph()
with graph.as_default():
    with gfile.FastGFile(graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def, name='')


# Saving the Graph to TensorBoard

Now that we've got the Inception graph saved in a Python variable, we can export it by using the `tf.train.SummaryWriter` class, which writes out data to disk for use by TensorBoard. We're not doing much other than saving the graph, so we'll just open up the writer and immediately close it. 

In [9]:
# Directory to save TensorBoard data
logdir = 'tensorboard/inception'

In [10]:
writer = tf.summary.FileWriter(logdir, graph=graph)
writer.close()

In a console, run the following command:

```bash
$ tensorboard --logdir='tensorboard/inception' --port=6006
```

By default, this command will start a TensorBoard server on `localhost:6006`. Once it's running, open up `localhost:6006` in your browser.

If you get an error stating that the port is in use, simply change the port number in the command.